In [79]:
import csv
import re
import spacy
from spacy import displacy

from owlready2 import *
import rdflib

import re

# pre-processing
def Filter(senSet):
    #remove content between [ ]
    print("Pre-processing...")
    for index in range(len(senSet)):
        while senSet[index].find('[')>=0:
            i_start = senSet[index].find('[')
            i_end = senSet[index].find(']')
            s = senSet[index][i_start:i_end+2]
            senSet[index] = senSet[index].replace(s, "")
            
# stopwords from parsing the whole sentence
def RemoveStopword1(phrase, doc, chunkStart, chunkEnd, stopList):
    result = phrase
    i_stop=0
    #start = chunk.start# to eliminate the condition when the first word of chunk is stop word
    for i_sen in range(chunkStart, chunkEnd):
        while i_stop < len(stopList) and stopList[i_stop] < i_sen-1:
            #print(str(stopList[i_stop]) + ' ' + str(i_sen))
            i_stop = i_stop+1
        # there is no stop word in current chunk
        if i_stop >= len(stopList):
            break;
        #print(i_sen)
        # finish going through the chunk
        if stopList[i_stop] > chunk.end-1:
            break
        # find the stop word and remove it
        if stopList[i_stop] == i_sen-1:
            #print(doc[i_sen-1])
            if i_sen-1 == chunkStart:
                result = result.replace(doc[i_sen-1].text + ' ', '')
                chunkStart = chunkStart+1
            else:
                result = result.replace(' ' + doc[i_sen-1].text, '')
    return result

# stopwords from parsing triple separately
def RemoveStopword2(inputPhrase):
    result = ''
    doc_phrase = nlp(str(inputPhrase))
    for token in doc_phrase:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #       token.shape_, token.is_alpha, token.is_stop)
        if not token.is_stop:
            result = result + token.text + ' '
        #else:
        #    print(token.text + ', ', end = '')    
    return result


# extract one triple from given sentence
def ExtractTriple(sen):
    # initialize the triple and stop word list
    subj = ""
    pred = ""
    obj = ""
    stopList = []
    
    # parse sentence
    doc = nlp(str(sen))
    print('\n' + str(index) + ': ' + senSet[index])
    
    ## visualize the semantic tree
    #options = {'compact': True, 'color': 'blue'}
    #displacy.serve(doc, style='dep', options=options)
    #displacy.serve(doc, style='dep')

    print('stop words: ', end='')
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #      token.shape_, token.is_alpha, token.is_stop)

        # record the index of stop words
        if token.is_stop:
            print(token.text + ', ', end='')
            stopList.append(token.i)
        if re.match('nsubj', token.dep_):   
            subj = token.text
        if re.match('ROOT', token.dep_): 
            pred = token.lemma_
            pred_orig = token.text
        if re.match('dobj', token.dep_): 
            obj = token.text
            '''#an earlier solution that I find not necessary
            obj = token.lemma_
            # to avoid cases like "-PRON-"
            if obj[0] == '-':
                obj = token.text'''
    print('\n')

    subj_1 = subj
    obj_1 = obj
    # using chunk to update subject and object
    for chunk in doc.noun_chunks:
        if chunk.root.head.text == pred_orig and re.match('nsubj', chunk.root.dep_):
            subj = chunk.text
            # remove stop words
            subj_1 = RemoveStopword1(subj, doc, chunk.start, chunk.end, stopList)

        if chunk.root.head.text == pred_orig and re.match('dobj|attr', chunk.root.dep_):
            obj = chunk.text
            # remove stop words
            obj_1 = RemoveStopword1(obj, doc, chunk.start, chunk.end, stopList)
        #print(chunk.text + ' ' + str(chunk.start))
        #print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

    print('Before : ' + subj + ' - ' + pred + ' - ' + obj)
    print('Method1: ' + subj_1 + ' - ' + pred + ' - ' + obj_1)

    # second method to remove stop words
    subj_2 = RemoveStopword2(subj)
    obj_2 = RemoveStopword2(obj)
    print('Method2: ' + subj_2 + '- ' + pred + ' - ' + obj_2 + '\n')

    return [subj, pred, obj]

# transfer a phrase to a URI form
def FormatURI(phrase):
    #print('Before formatting:  ' + phrase)
    chars = list(phrase)
    if len(chars) > 0:
        chars[0] = chars[0].upper()
    for i in range(len(chars)):
        if chars[i] == ' ' and i+1 < len(chars):
            chars[i+1] = chars[i+1].upper()
    phrase = ''.join(chars)
    phrase = phrase.replace(' ', '')
    phrase = re.sub(r'[^a-zA-Z0-9\s]', '', phrase)
    print('After formatting:  ' + phrase)
    return phrase

# query the given triple in the ontology with SPARQL
# return true/false as result
def QueryTriple(subj, pred, obj):
    prefix = """
    PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpd:<http://dbpedia.org/ontology/>
    """
    #subj = "provinceLink"
    #pred = "range"
    #obj = "Province"
    qSelect = prefix + """
    SELECT ?sub WHERE {
      ?sub rdf:""" + FormatURI(pred) + """ dbpd:""" + FormatURI(obj) + """.
    }"""
    qAsk = prefix + """
    ASK {
        dbpd:""" + FormatURI(subj) + """ rdf:""" + FormatURI(pred) + """ dbpd:""" + FormatURI(obj) + """.
    }"""
    
    r = list(m_graph.query(qAsk))
    return r

# load Spacy NLP dictionary
nlp = spacy.load('en_core_web_sm')

# load DBPD ontology and construct graph for query
m_world = World()# Owlready2 stores every triples in a ‘World’ object
m_onto = m_world.get_ontology("dbpedia.owl").load()
m_graph = m_world.as_rdflib_graph()

# load data
file = open("shortdataset.csv", "r")
#file = open("newdataset_formatted.csv", "r")
reader = csv.reader(file)
senSet = []
for item in reader:
    #format sentences in item as string
    fullP = "".join(item)
    splitP = fullP.split(";", 3);
    splitS = splitP[3][1:len(splitP[3])].split(".");
    #print(splitS)
    for sen in splitS:
        senSet.append(sen)#store the sentence into an array
file.close()
print("Total sentences: " + str(len(senSet)))

# pre-processing
Filter(senSet)

# parse and query each sentence
for index in range(len(senSet)):
    #index = 13
    
    # extract triple from current sentence
    [subj, pred, obj] = ExtractTriple(senSet[index])
    print('After : ' + subj + ' - ' + pred + ' - ' + obj)

    # query the triple in dbpd with SPARQL
    queryResult = QueryTriple(subj, pred, obj)
    print('Triple Query Result: ' + str(queryResult))
    
    

* Owlready2 * WARNING: http://dbpedia.org/ontology/senator belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/productShape belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/latinName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataPrope

Total sentences: 46
Pre-processing...

0: Sure um start off - I'm a Purdue grad
stop words: off, a, 

Before : I - be - a Purdue grad
Method1: I - be - Purdue grad
Method2: I - be - Purdue grad 

After : I - be - a Purdue grad
After formatting:  Be
After formatting:  APurdueGrad
After formatting:  I
After formatting:  Be
After formatting:  APurdueGrad
Triple Query Result: [False]

1:   Um my title is director of landscape architecture so um  for the ah staff of people
stop words: my, is, of, so, for, the, of, 

Before : my title - be - director
Method1: title - be - director
Method2: title - be - director 

After : my title - be - director
After formatting:  Be
After formatting:  Director
After formatting:  MyTitle
After formatting:  Be
After formatting:  Director
Triple Query Result: [False]

2:   Um we manage all the construction and innovation projects for the university across the state
stop words: we, all, the, and, for, the, across, the, 

Before : we - manage - all the construct


20:   Um its its variance um with good design you may be able to integrate that with a resting interval uh get up halfway and then you push out um and then you go back up to go on up to what would be the the larger tree house space
stop words: its, its, with, you, may, be, to, that, with, a, get, up, and, then, you, out, and, then, you, go, back, up, to, go, on, up, to, what, would, be, the, the, 

Before : you - go - that
Method1: you - go - that
Method2: - go - 

After : you - go - that
After formatting:  Go
After formatting:  That
After formatting:  You
After formatting:  Go
After formatting:  That
Triple Query Result: [False]

21:   Um this is what I do day in and day out
stop words: this, is, what, do, in, and, out, 

Before : I - be - what
Method1: I - be - what
Method2: I - be - 

After : I - be - what
After formatting:  Be
After formatting:  What
After formatting:  I
After formatting:  Be
After formatting:  What
Triple Query Result: [False]

22:   Um talk to designers and drea


42: zip line and things like that
stop words: and, that, 

Before :  - line - 
Method1:  - line - 
Method2: - line - 

After :  - line - 
After formatting:  Line
After formatting:  
After formatting:  
After formatting:  Line
After formatting:  
Triple Query Result: [False]

43:  So for something like this where we want the tree house to be open and accessible we would have to be able to - for one safety purposes for for kids that are up there close off that opening when we're not running it um but two also for that access prevention
stop words: for, something, this, where, we, the, to, be, and, we, would, have, to, be, to, for, one, for, for, that, are, up, there, off, that, when, we, not, it, but, two, also, for, that, 

Before : we - have - it
Method1: we - have - it
Method2: - have - 

After : we - have - it
After formatting:  Have
After formatting:  It
After formatting:  We
After formatting:  Have
After formatting:  It
Triple Query Result: [False]

44: 
stop words: 

Before :  - 

In [49]:
# to remove WARNINGs from Owlready2
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')


In [59]:
# reference: https://pythonhosted.org/Owlready2/world.html
from owlready2 import *
import rdflib

my_world = World()# Owlready2 stores every triples in a ‘World’ object
onto = my_world.get_ontology("dbpedia.owl").load()

graph = my_world.as_rdflib_graph()
print(len(graph))

prefix = """
PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbpd:<http://dbpedia.org/ontology/>
"""

'''r = list(graph.query(prefix + """
SELECT ?sub WHERE {
  ?sub rdf:range  dbpd:Province.
}"""))'''

'''r = list(graph.query(prefix + """
ASK {
  dbpd:provinceLink rdf:range  dbpd:Province.
}"""))'''

r = list(graph.query(prefix + """
DESCRIBE ?sub WHERE {
  ?sub rdf:range  dbpd:Province.
}"""))

print(r)

* Owlready2 * WARNING: http://dbpedia.org/ontology/senator belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/productShape belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/latinName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...


31050


* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6393Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6392Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/ingredientName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/greekName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...


Exception: DESCRIBE not implemented